In [1]:
import pandas as pd
import httplib2
from urllib import parse
import json
from datetime import datetime, timedelta

### CSV 로딩

In [180]:
csv_result = pd.read_csv('./csv/test_prd_20171108.csv')
csv_tot_rows = len(csv_result)
csv_tot_rows

29940

### 루프 Index 배열 생성
- loop : 조회의 시작번호로 0으로 입력시, 첫번째 인덱스부터 데이터를 조회한다.
- block_size : import한 CSV의 행수를 지정한다.

In [36]:
# 예를 들어 전체데이터를 400개씩 인덱싱할 경우, loop=0, block_size=400이 된다.
def create_idx(tot_rows, block_size = 400):
    loop = 0
    idx_arr = []    
    
    for i in range(tot_rows):
        idx = i % block_size

        if idx == 0:
            if i !=0:
                #print(loop*block_size, i-1)
                idx_arr.append({'start':loop*block_size,'end':i-1})
                loop = loop+1
        elif i == len(csv_result)-1:
            #print(loop*block_size, i)
            idx_arr.append({'start':loop*block_size,'end':i})
    
    return idx_arr

[{'start': 0, 'end': 399}, {'start': 400, 'end': 799}, {'start': 800, 'end': 1199}, {'start': 1200, 'end': 1599}, {'start': 1600, 'end': 1999}, {'start': 2000, 'end': 2399}, {'start': 2400, 'end': 2799}, {'start': 2800, 'end': 3199}, {'start': 3200, 'end': 3599}, {'start': 3600, 'end': 3999}, {'start': 4000, 'end': 4399}, {'start': 4400, 'end': 4799}, {'start': 4800, 'end': 5199}, {'start': 5200, 'end': 5599}, {'start': 5600, 'end': 5999}, {'start': 6000, 'end': 6399}, {'start': 6400, 'end': 6799}, {'start': 6800, 'end': 7199}, {'start': 7200, 'end': 7599}, {'start': 7600, 'end': 7999}, {'start': 8000, 'end': 8399}, {'start': 8400, 'end': 8799}, {'start': 8800, 'end': 9199}, {'start': 9200, 'end': 9599}, {'start': 9600, 'end': 9999}, {'start': 10000, 'end': 10399}, {'start': 10400, 'end': 10799}, {'start': 10800, 'end': 11199}, {'start': 11200, 'end': 11599}, {'start': 11600, 'end': 11999}, {'start': 12000, 'end': 12399}, {'start': 12400, 'end': 12799}, {'start': 12800, 'end': 13199}, 

# 네이버 API 사용을 위한 토큰 세팅

In [37]:
naver_app_keys = [
    {'client_id':'CMHEay4zZQlBUNvIwIRS','client_secret':'gOF9laz3by'},
    {'client_id':'9bpdvOmcRsJVQGjPPcGe','client_secret':'lArvjy7M9H'},
    {'client_id':'j2Y5_SaIAaAJzCCy7162','client_secret':'_w3ZaRFGnl'},
    {'client_id':'IPsWTOFJFQ3mmqsu5tJL','client_secret':'FTs9FlzRji'},
    {'client_id':'jkTyIyd8NFAAyyODYKdM','client_secret':'Dsvl5hHpbf'}]

In [38]:
def getUrl(sort,query):
    url = 'https://openapi.naver.com/v1/search/shop.json?display=1&start=1&sort=' + sort + '&query=' + parse.quote(query)
    
    print(url)
    return url

# HTTP 통신 함수
### json 리턴 데이터를 검증용 CSV로 재변환

In [144]:
# 네이버 가격 비교 호출시 높은 가격 순, 낮은 가격 순으로 모두 호출하여 상품코드별 데이터로 재정렬
def common_http(sort, req_data, headers, arr_to_append = []):
    
    arr_to_append.append("title")
    arr_to_append.append(1000)
    arr_to_append.append("link")
    if sort == 'asc':
        arr_to_append.append(0)
    
    return arr_to_append
#     http = httplib2.Http()
        
#     response,content = http.request(getUrl(sort,req_data['GOODS_NAME2']), 'GET', headers=headers)
    
    
    
#     if response.status == 200:
#         text = content.decode('utf-8')
#         data = json.loads(text)
        
#         if len(data['items']) > 0:
#             # 네이버 조회 상품명
#             arr_to_append.append(data['items'][0]['title'])
#             # 네이버 가격
#             arr_to_append.append(data['items'][0]['lprice'])
#             # 네이버 상품 링크
#             arr_to_append.append(data['items'][0]['link'])
#             # 가격비율
#             if sort == 'asc':
#                 percentage = (int(data['items'][0]['lprice']) / int(req_data['KHS_PRICE'])) * 100
#                 arr_to_append.append(percentage)
#         else:
#             arr_to_append.append('no_title')
#             arr_to_append.append('0')
#             arr_to_append.append('no_link')
#             if sort == 'asc':
#                 arr_to_append.append('0')
#         return arr_to_append    

### 네이버 가격 비교를 호출하여 리턴하는 공통 함수

In [145]:
# 네이버 앱 키
naver_app_keys = [
{'client_id':'CMHEay4zZQlBUNvIwIRS','client_secret':'gOF9laz3by'},
{'client_id':'9bpdvOmcRsJVQGjPPcGe','client_secret':'lArvjy7M9H'},
{'client_id':'j2Y5_SaIAaAJzCCy7162','client_secret':'_w3ZaRFGnl'},
{'client_id':'IPsWTOFJFQ3mmqsu5tJL','client_secret':'FTs9FlzRji'},
{'client_id':'jkTyIyd8NFAAyyODYKdM','client_secret':'Dsvl5hHpbf'}
]

In [165]:
def req_naver_price(req_data, token_idx):
    
    # 헤더설정
    headers = {
        "X-Naver-Client-Id": naver_app_keys[token_idx]['client_id'],
        "X-Naver-Client-Secret": naver_app_keys[token_idx]['client_secret'],
        "Content-Type": 'application/json'
    }
    
    # 가격 데이터를 담는 임시배열
    temp_arr = []
    
    # 네이버가격비교할 노출상품명, 상품코드
    prd_cls_lnm = req_data['PRD_CLS_LNM']
    query = req_data['GOODS_NAME2']
    khs_price = req_data['KHS_PRICE']
    prd_cd = req_data['PRD_CD']
    
    # Key로 사용할 기준 상품정보를 먼저 담아 놓는다.
    temp_arr.append(prd_cls_lnm)
    temp_arr.append(query)
    temp_arr.append(khs_price)
    temp_arr.append(prd_cd)
    
    # 오름차순(낮은 가격 추출)
    result_arr_asc = common_http('asc', req_data, headers, [])
    
    # 내림차순(높은 가격 추출)
    result_arr = common_http('dsc', req_data, headers, result_arr_asc)

    return temp_arr + result_arr

### 400개씩 쪼개서 전송

In [233]:
def arrange_data(result_arr = []):
    
    columns = ['대분류','EC노출상품명','판매가','상품코드','최저가상품명','최저가','최저가링크','최저가비율(GS대비)','최고가상품','최고가','최고가링크']
    
    # block 갯수
    idx_arr = create_idx(csv_tot_rows, block_size=400)
    
    # 블록 인덱스 : 0이면 0~399, 1이면 400~799, ...
    idx_num=0
    
    token_idx = 0
    
    block_buffer = 0
    
    # len(idx_arr)은 리턴될 데이터의 행수와 같다.
    for i in range(len(idx_arr)):
            
        # 특정행 선택하여 쿼리
        temp_data = csv_result.loc[idx_arr[idx_num]['start']:idx_arr[idx_num]['end']]
        
        # API 호출 결과를 담는 배열
        arranged_arr = []

        # 네이버 API 400개 호출 : 병렬로 호출하는 방식으로 수정 필요
        for j in range(len(temp_data)):
            req_data = temp_data.loc[(j+int(idx_arr[idx_num]['start']))]

            # 네이버 API 호출
            api_result = req_naver_price(req_data, token_idx)
            # print("api_result", api_result)
            arranged_arr.append(api_result)
        
        
        # 통신이 끝난 후 400개를 DataFrame에 담는다.
        temp_df = pd.DataFrame(arranged_arr, columns=columns)
        
        # 최종 결과 파일로 쓸 arr에 800개를 담는다.
        result_arr.append(temp_df)
        
        idx_num = idx_num + 1
        
        if i > 0 and (i % 15) == 0:
            token_idx = token_idx + 1
            
    # 최종 결과 파일 UNION ALL
    result = pd.concat(result_arr)
    
    return result

In [234]:
final_result_arr_to_csv = arrange_data([])

i :  15
rest :  0
cur_key :  {'client_id': '9bpdvOmcRsJVQGjPPcGe', 'client_secret': 'lArvjy7M9H'}
token_idx :  1
i :  30
rest :  0
cur_key :  {'client_id': 'j2Y5_SaIAaAJzCCy7162', 'client_secret': '_w3ZaRFGnl'}
token_idx :  2
i :  45
rest :  0
cur_key :  {'client_id': 'IPsWTOFJFQ3mmqsu5tJL', 'client_secret': 'FTs9FlzRji'}
token_idx :  3
i :  60
rest :  0
cur_key :  {'client_id': 'jkTyIyd8NFAAyyODYKdM', 'client_secret': 'Dsvl5hHpbf'}
token_idx :  4


In [183]:
final_result_arr_to_csv

,대분류,EC노출상품명,판매가,상품코드,최저가상품명,최저가,최저가링크,최저가비율(GS대비),최고가상품,최고가,최고가링크
0,가구/침구,1300K Mariage pink cushion,58000,23358291,title,1000,link,0,title,1000,link
1,가구/침구,내츄럴힐링양털차렵패드 Q,79900,23373289,title,1000,link,0,title,1000,link
2,가구/침구,까사미아 까사온 스칸센 원목 1200 테이블,179300,23371383,title,1000,link,0,title,1000,link
3,가구/침구,1200M 내추럴코튼 피그먼트 차렵이불세트 Q,64900,23357820,title,1000,link,0,title,1000,link
4,가구/침구,1200M 카리노 3단 벽선반,176000,23358495,title,1000,link,0,title,1000,link
5,가구/침구,1200M 스페이스쉽 DTP암막커튼 140X225,29800,23357855,title,1000,link,0,title,1000,link
6,가구/침구,1300K 모던다이아 베이지 극세사 러그 200x260cm,112500,23359122,title,1000,link,0,title,1000,link
7,가구/침구,1300K 로즈니아 극세사 침대패드 3color SQ 싱글패드단품,86000,23359128,title,1000,link,0,title,1000,link
8,가구/침구,1200M 밸런스가구 에쉬 원목 B타입 3인용 가죽 소파,895000,23358394,title,1000,link,0,title,1000,link
9,가구/침구,1300K MACLED 화이트 포르테 실내등만,23400,23359659,title,1000,link,0,title,1000,link


In [ ]:
# 엑셀 파일 생성
result.to_excel('result_data_naver_100.xlsx', sheet_name='naver-price')